# DS 2002 Final Project
## Nina Jannatifar (worked individually)
### Questions that users can ask:
1. Top 10 Shows of 2020
2. Top 10 Movies of 2020
3. Genres of the Top 5 Movies of 2019
4. Genres of the Top 5 Shows of 2019
5. Number of Seasons of the Top Show of 2014
6. Top Horror Movie of All Time
7. Country Producing the Top Movie of 2010
8. Duration of the Top Movie of 2000
9. Top 10 Years with the Highest Average Movie IMDB Scores
10. Age Certification of Top Show of 2016

In [1]:
# Set up WD 
import os
#os.chdir(r'/Users/ninij/Desktop')

In [2]:
# Connecting MongoDB to load db from Part 1
import pymongo
host_name = "localhost"
port = "27017"

conn_str = {
    "local" : f"mongodb://{host_name}:{port}/Netflix",
}
client = pymongo.MongoClient(conn_str["local"])
db = client.Netflix
showData = db.Shows
movieData = db.Movies

In [3]:
# Machine Learning portion
import nltk #natural language processor for python
nltk.download('punkt') #information for parsing words

from nltk import word_tokenize,sent_tokenize # tokenizer pulls words from what user provides

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#read more on the steamer https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8
# stemmer program processes for tensorflow

# need to pip install nltk, tflearn, and tensorflow
# pickle - another tokenizer for taking out sentences and words
import numpy as np 
import tflearn # for tensorflow
import tensorflow as tf
import random
import json
import pickle

# Open the intents file
# Intents file is how to create pattern 
#notes for intents file -> create tag for question 1; 

### __________________don't edit___________________
with open("intents.json") as file:
    data = json.load(file)

# create a pickle file 
try:
    with open("data.pickle","rb") as f:
        words, labels, training, output = pickle.load(f)

        
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])


    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
               bag.append(1)
            else:
              bag.append(0)
    
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
        
        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)
    
    with open("data.pickle","wb") as f:
        pickle.dump((words, labels, training, output), f)

# training so that if the user's input doesn't match, it will take a guess
# learns based on intents file

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")


def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return np.array(bag)
### __________________don't edit___________________



Training Step: 17999  | total loss: 0.00014 | time: 0.075s
| Adam | epoch: 1000 | loss: 0.00014 - acc: 1.0000 -- iter: 136/143
Training Step: 18000  | total loss: 0.00014 | time: 0.078s
| Adam | epoch: 1000 | loss: 0.00014 - acc: 1.0000 -- iter: 143/143
--
INFO:tensorflow:/Users/ninij/Desktop/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from /Users/ninij/Desktop/model.tflearn


In [4]:
# Prepare the chat function and pull responses from database collections
def chat():
    print("\nStart talking with the bot!(type quit to stop)")
    print("Here are the questions you can ask me:\n1. Top 10 Shows of 2020\n2. Top 10 Movies of 2020\n3. Genres of the Top 5 Movies of 2019")
    print("4. Genres of the Top 5 Shows of 2019\n5. Number of Seasons of the Top Show of 2014\n6. Top Horror Movie of All Time\n7. Country Producing the Top Movie of 2010")
    print("8. Duration of the Top Movie of 2000\n9. Top 10 Years with the Highest Average Movie IMDB Scores\n10. Age Certification of Top Show of 2016")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        result = model.predict([bag_of_words(inp, words)])[0]
        result_index = np.argmax(result)
        tag = labels[result_index]

# checking for a result with a confidence level greater than 70% based on the training
# predicts how clearly the response will answer the question
# this is where you figure out what question was asked - here it randomly selects response
        if result[result_index] > 0.7:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    if tag == "question1":
                        q1data = showData.find({"RELEASE_YEAR":2020},{"_id":0,"DURATION":0,"NUMBER_OF_SEASONS":0,"MAIN_GENRE":0,"MAIN_PRODUCTION":0,"RELEASE_YEAR":0,"SCORE":0,"AGE_CERTIFICATION":0}).limit(10).sort([("SCORE", pymongo.DESCENDING)])
                        print(" \nThe top 10 shows of 2020 are:")
                        for x in q1data:
                            print(x)
                        print("")
                    elif tag == "question2":
                        q2data = movieData.find({"RELEASE_YEAR":2020},{"_id":0,"DURATION":0,"NUMBER_OF_SEASONS":0,"MAIN_GENRE":0,"MAIN_PRODUCTION":0,"RELEASE_YEAR":0,"SCORE":0,"AGE_CERTIFICATION":0}).limit(10).sort([("SCORE", pymongo.DESCENDING)])
                        print("\nThe top 10 movies of 2020 are:")
                        for x in q2data:
                            print(x)
                        print("")
                    elif tag == "question3":
                        q3data = movieData.find({"RELEASE_YEAR":2019},{"_id":0,"DURATION":0,"NUMBER_OF_SEASONS":0,"MAIN_PRODUCTION":0,"AGE_CERTIFICATION":0,"RELEASE_YEAR":0,"SCORE":0, "TITLE":0}).limit(5).sort([("SCORE", pymongo.DESCENDING)])
                        print("\nThe genres of the top 5 movies of 2019 are:")
                        for x in q3data:
                            print(x)
                        print("")
                    elif tag == "question4":
                        q4data = showData.find({"RELEASE_YEAR":2019},{"_id":0,"DURATION":0,"NUMBER_OF_SEASONS":0,"MAIN_PRODUCTION":0,"AGE_CERTIFICATION":0,"RELEASE_YEAR":0,"SCORE":0, "TITLE":0}).limit(5).sort([("SCORE", pymongo.DESCENDING)])
                        print("\nThe genres of the top 5 shows of 2019 are:")
                        for x in q4data:
                            print(x)
                        print("")
                    elif tag == "question5":
                        q5data = showData.find({"RELEASE_YEAR":2014},{"_id":0,"DURATION":0,"MAIN_PRODUCTION":0,"AGE_CERTIFICATION":0,"RELEASE_YEAR":0,"SCORE":0, "TITLE":0,"MAIN_GENRE":0}).limit(1).sort([("SCORE", pymongo.DESCENDING)])
                        print("\nThe number of seasons in the top show of 2014 is:")
                        for x in q5data:
                            print(x)
                        print("The name of this show is:")
                        q5data2 = showData.find({"RELEASE_YEAR":2014},{"_id":0,"DURATION":0,"MAIN_PRODUCTION":0,"AGE_CERTIFICATION":0,"RELEASE_YEAR":0,"SCORE":0, "NUMBER_OF_SEASONS":0,"MAIN_GENRE":0}).limit(1).sort([("SCORE", pymongo.DESCENDING)])
                        for x in q5data2:
                            print(x)
                        print("")
                    elif tag == "question6":
                        q6data = movieData.find({"MAIN_GENRE":"horror"},{"_id":0,"RELEASE_YEAR":0,"SCORE":0, "NUMBER_OF_SEASONS":0,"MAIN_GENRE":0,"DURATION":0,"MAIN_PRODUCTION":0,"AGE_CERTIFICATION":0}).limit(1).sort([("SCORE", pymongo.DESCENDING)])
                        print("\nThe best horror movie according to IMDB scores is:")
                        for x in q6data:
                            print(x)
                        print("")
                    elif tag == "question7":
                        q7data = movieData.find({"RELEASE_YEAR":2010},{"_id":0,"DURATION":0,"TITLE":0,"AGE_CERTIFICATION":0,"RELEASE_YEAR":0,"SCORE":0, "NUMBER_OF_SEASONS":0,"MAIN_GENRE":0}).limit(1).sort([("SCORE", pymongo.DESCENDING)])
                        print("\nThe country that produced the top movie of 2010 is:")
                        for x in q7data:
                            print(x)
                        q7data2 = movieData.find({"RELEASE_YEAR":2010},{"_id":0,"DURATION":0,"MAIN_PRODUCTION":0,"AGE_CERTIFICATION":0,"RELEASE_YEAR":0,"SCORE":0, "NUMBER_OF_SEASONS":0,"MAIN_GENRE":0}).limit(1).sort([("SCORE", pymongo.DESCENDING)])
                        print("The name of this movie is:")
                        for x in q7data2:
                            print(x)
                        print("")
                    elif tag == "question8":
                        q8data = movieData.find({"RELEASE_YEAR":2000},{"_id":0,"MAIN_PRODUCTION":0,"AGE_CERTIFICATION":0,"RELEASE_YEAR":0,"SCORE":0, "NUMBER_OF_SEASONS":0,"MAIN_GENRE":0,"TITLE":0}).limit(1).sort([("SCORE", pymongo.DESCENDING)])
                        print("\nThe duration of the top movie in 2000 is:")
                        for x in q8data:
                            print(x)
                        q8data2 = movieData.find({"RELEASE_YEAR":2000},{"_id":0,"MAIN_PRODUCTION":0,"AGE_CERTIFICATION":0,"RELEASE_YEAR":0,"SCORE":0, "NUMBER_OF_SEASONS":0,"MAIN_GENRE":0,"DURATION":0}).limit(1).sort([("SCORE", pymongo.DESCENDING)])
                        print("The name of this movie is:")
                        for x in q8data2:
                            print(x)
                        print("")
                    elif tag == "question9":
                        q9data = movieData.aggregate([{"$group":{"_id":"$RELEASE_YEAR", "avgScore": {"$avg":"$SCORE"}}}, 
                               {"$sort" : {"avgScore" : -1}},
                              {"$limit" : 10}])
                        print("\nThe top 10 years with the highest average IMDB scores are:")
                        for x in q9data:
                            print(x)
                        print("")
                    elif tag == "question10":
                        q10data = showData.find({"RELEASE_YEAR":2016},{"_id":0,"MAIN_PRODUCTION":0,"RELEASE_YEAR":0,"SCORE":0, "NUMBER_OF_SEASONS":0,"MAIN_GENRE":0,"DURATION":0, "TITLE":0}).limit(1).sort([("SCORE", pymongo.DESCENDING)])
                        print("\nThe age certification of the top show in 2016 is:")
                        for x in q10data:
                            print(x)
                        q10data2 = showData.find({"RELEASE_YEAR":2016},{"_id":0,"MAIN_PRODUCTION":0,"RELEASE_YEAR":0,"SCORE":0, "NUMBER_OF_SEASONS":0,"MAIN_GENRE":0,"DURATION":0, "AGE_CERTIFICATION":0}).limit(1).sort([("SCORE", pymongo.DESCENDING)])
                        print("The name of this show is:")
                        for x in q10data2:
                            print(x)
                        print("")
# won't need to hard code responses -> just match tags and then pull appropriately from mongo; basically make elif statements
        else:
            print("I didnt get that. Please ask again.")
            print("Here are the questions you can ask me:\n1. Top 10 Shows of 2020\n2. Top 10 Movies of 2020\n3. Genres of the Top 5 Movies of 2019")
            print("4. Genres of the Top 5 Shows of 2019\n5. Number of Seasons of the Top Show of 2014\n6. Top Horror Movie of All Time\n7. Country Producing the Top Movie of 2010")
            print("8. Duration of the Top Movie of 2000\n9. Top 10 Years with the Highest Average Movie IMDB Scores\n10. Age Certification of Top Show of 2016")
            print("")

In [5]:
#Run the chatbot
chat()


Start talking with the bot!(type quit to stop)
Here are the questions you can ask me:
1. Top 10 Shows of 2020
2. Top 10 Movies of 2020
3. Genres of the Top 5 Movies of 2019
4. Genres of the Top 5 Shows of 2019
5. Number of Seasons of the Top Show of 2014
6. Top Horror Movie of All Time
7. Country Producing the Top Movie of 2010
8. Duration of the Top Movie of 2000
9. Top 10 Years with the Highest Average Movie IMDB Scores
10. Age Certification of Top Show of 2016
You: top shows 2020
 
The top 10 shows of 2020 are:
{'TITLE': 'The Last Dance'}
{'TITLE': 'The Last Dance'}
{'TITLE': "The Queen's Gambit"}
{'TITLE': "It's Okay to Not Be Okay"}
{'TITLE': "It's Okay to Not Be Okay"}
{'TITLE': "The Queen's Gambit"}
{'TITLE': 'Caliphate'}
{'TITLE': 'Itaewon Class'}
{'TITLE': 'Caliphate'}
{'TITLE': 'Itaewon Class'}

You: What are the top 2020 shows?
 
The top 10 shows of 2020 are:
{'TITLE': 'The Last Dance'}
{'TITLE': 'The Last Dance'}
{'TITLE': "The Queen's Gambit"}
{'TITLE': "It's Okay to Not 

# References

https://www.mongodb.com/docs/manual/reference/operator/aggregation/limit/

https://www.mongodb.com/docs/manual/reference/operator/aggregation/sort/

https://www.geeksforgeeks.org/aggregation-in-mongodb/